In [1]:
import cv2 as cv
import numpy as np
import os
from PIL import Image
from scipy import spatial
import glob

In [15]:
# Load gambar utama
main_image = cv.imread("MainIMG.jpg")

# Lakukan restorasi digital untuk menghilangkan noise
denoised_image = cv.bilateralFilter(main_image, 4, 45, 65)
darker_image = cv.subtract(denoised_image, (10, 10, 10, 0))


# Tampilkan gambar hasil
cv.imshow("Original Image", main_image)
cv.imshow("Denoised Image", denoised_image)
cv.imshow("Sharpened Image", darker_image)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite("Darker_Image.jpg", darker_image)


True

In [3]:

sample_image = cv.imread("SampleIMG.jpg")

# Lakukan restorasi digital untuk menghilangkan noise
denoised_image_1 = cv.bilateralFilter(sample_image, 5, 50, 50)
ligther_image = cv.add(denoised_image_1, (10, 10, 10, 0))


# Tampilkan gambar hasil
cv.imshow("Original Image", sample_image)
cv.imshow("Denoised Image", denoised_image_1)
cv.imshow("Sharpened Image", ligther_image)
cv.waitKey(0)
cv.destroyAllWindows()

In [4]:
import os

def process_image(image_path):
    sample_image = cv.imread(image_path)
    
    resized_image = cv.resize(sample_image, (25, 34))
    
    denoised_image = cv.bilateralFilter(resized_image, 5, 50, 50)
    
    lighter_image = cv.add(denoised_image, (10, 10, 10, 0))

    return lighter_image

folder_path = "sample"

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".JPG"): 

        image_path = os.path.join(folder_path, filename)
        
        processed_image = process_image(image_path)
        
        cv.imshow("Original Image", cv.imread(image_path))
        cv.imshow("Processed Image", processed_image)
        cv.waitKey(0)

# Tutup semua jendela gambar
cv.destroyAllWindows()


In [5]:
folder_path = "sample"

# Buat folder untuk menyimpan gambar yang diresize
output_folder_path = "SampleResize"
os.makedirs(output_folder_path, exist_ok=True)

# Loop melalui setiap gambar di dalam folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".JPG"): 
        # Path lengkap gambar
        image_path = os.path.join(folder_path, filename)
        
        # Proses gambar
        processed_image = process_image(image_path)
        
        # Simpan gambar yang diresize ke dalam folder baru
        output_path = os.path.join(output_folder_path, filename)
        cv.imwrite(output_path, processed_image)
        print(f"Image saved: {output_path}")

print("All images processed and saved successfully.")

Image saved: SampleResize\Sample_0.JPG
Image saved: SampleResize\Sample_1.jpg
Image saved: SampleResize\Sample_10.jpg
Image saved: SampleResize\Sample_11.jpg
Image saved: SampleResize\Sample_12.jpg
Image saved: SampleResize\Sample_13.jpg
Image saved: SampleResize\Sample_2.jpg
Image saved: SampleResize\Sample_3.jpg
Image saved: SampleResize\Sample_4.jpg
Image saved: SampleResize\Sample_5.JPG
Image saved: SampleResize\Sample_6.jpg
Image saved: SampleResize\Sample_7.jpg
Image saved: SampleResize\Sample_8.jpg
Image saved: SampleResize\Sample_9.jpg
All images processed and saved successfully.


In [143]:
main_photo_path = "Darker_Image.jpg"
tile_photos_path = "SampleResize"
tile_size = (4, 6)

tile_paths = []
for file in glob.glob(str(tile_photos_path)):  # Convert Path object to string
    tile_paths.append(file)


# Import and resize all tiles
tiles = []
for path in glob.glob("SampleResize/*"):
    tile = Image.open(path)
    tile = tile.resize(tile_size)
    tiles.append(tile)


# Calculate dominant color
colors = []
for tile in tiles:
    mean_color = np.array(tile).mean(axis=0).mean(axis=0)
    colors.append(mean_color)


In [144]:
main_photo = Image.open(main_photo_path)

width = int(np.round(main_photo.size[0] / tile_size[0]))
height = int(np.round(main_photo.size[1] / tile_size[1]))

resized_photo = main_photo.resize((width, height))

In [145]:
tree = spatial.KDTree(colors)

# Empty integer array to store indices of tiles
closest_tiles = np.zeros((width, height), dtype=np.uint32)

for i in range(width):
	for j in range(height):
		pixel = resized_photo.getpixel((i, j))  # Getthe pixel color at (i, j)
		closest = tree.query(pixel)             # Returns (distance, index)
		closest_tiles[i, j] = closest[1]        # We only need the index

In [146]:
output = Image.new('RGB', main_photo.size)

# Draw tiles
for i in range(width):
	for j in range(height):
		# Offset of tile
		x, y = i*tile_size[0], j*tile_size[1]
		# Index of tile
		index = closest_tiles[i, j]
		# Draw tile
		output.paste(tiles[index], (x, y))

# Save output
output.save("output4.jpg")